## Notebook Objective
---

This notebook documents a class taught to the former AI President of Tesla, Andrej Karpathy, on building a GPT (Generative Pre-trained Transformer). The code presented is not my own, but rather an attempt to understand the technology in a more technical manner. Regarding the dataset, originally Shakespeare's text (Tiny Shakespeare) was used, but I chose to use "The Little Prince" to evaluate the results.

## What I Learned
---
During the class, I delved into the fascinating world of GPT technology and discovered that its essence is rooted in the concept of transformers, originating from the paper "Attention is All You Need." To build something as impressive as generating new reference texts, I learned that there are some key steps to follow:

- Choosing the Data: Carefully selecting the data is crucial. They will serve as the foundation upon which the GPT will learn and generate new content.

- Tokenizing the Data: This process involves breaking down the data into smaller parts, such as words or subwords, so that the neural network can understand and process them efficiently.

- Organizing the Data: The data needs to be organized in a way that the neural network can predict the next elements based on the patterns it identifies. For example, if the word "Hello" was encoded as 11 22 33 44 55, the code needs to be able to predict what will come after increasing combinations of these elements, such as 11 22, 11 22 33, and so on.

This progressive learning process allows the GPT to absorb information from the training data and subsequently generate new texts based on this solid foundation.

## Modeling
---

In [1]:
# read it in to inspect it
with open('/kaggle/input/pequeno-p-text/pequenop.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  77454


In [3]:
# let's look at the first 1000 characters
print(text[:1000])

I 

Certa vez, quando tinha seis anos, vi num livro sobre a Floresta Virgem, "Historias 
Vividas", uma imponente gravura. Representava ela uma jiboia que engolia uma fera. Eis 
a copia do desenho. 



Dizia o livro: "As jiboias engolem, sem mastigar, a presa inteira. Em seguida, nao 
podem mover-se e dormem os seis meses da digestao." 

Refleti muito entao sobre as aventuras da selva, e fiz, com lapis de cor, o meu 
primeiro desenho. Meu desenho numero 1 era assim: 



Mostrei minha obra-prima as pessoas grandes e perguntei se o meu desenho lhes 
fazia medo. 

Responderam-me: "Por que e que um chapeu faria medo?" 

Meu desenho nao representava um chapeu. Representava uma jiboia digerindo um 
elefante. Desenhei entao o interior da jiboia, a fim de que as pessoas grandes pudessem 
compreender. Elas tern sempre necessidade de explicates . Meu desenho numero 2 era 
assim: 



As pessoas grandes aconselharam-me deixar de lado os desenhos de jiboias abertas 
ou fechadas, e dedicar-me de pref

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"&'()*,-./012356789:;=>?ABCDEFGHIJLMNOPQRSTUVXZ\^abcdefghijlmnopqrstuvxyz¥—“”
80


In [5]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("chatgpt"))
print(decode(encode("chatgpt")))

[54, 59, 52, 70, 58, 66, 70]
chatgpt


In [6]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([77454]) torch.int64
tensor([35,  1,  0,  0, 29, 56, 68, 70, 52,  1, 72, 56, 75,  9,  1, 67, 71, 52,
        64, 55, 65,  1, 70, 60, 64, 59, 52,  1, 69, 56, 60, 69,  1, 52, 64, 65,
        69,  9,  1, 72, 60,  1, 64, 71, 63,  1, 62, 60, 72, 68, 65,  1, 69, 65,
        53, 68, 56,  1, 52,  1, 32, 62, 65, 68, 56, 69, 70, 52,  1, 47, 60, 68,
        58, 56, 63,  9,  1,  3, 34, 60, 69, 70, 65, 68, 60, 52, 69,  1,  0, 47,
        60, 72, 60, 55, 52, 69,  3,  9,  1, 71, 63, 52,  1, 60, 63, 66, 65, 64,
        56, 64, 70, 56,  1, 58, 68, 52, 72, 71, 68, 52, 11,  1, 43, 56, 66, 68,
        56, 69, 56, 64, 70, 52, 72, 52,  1, 56, 62, 52,  1, 71, 63, 52,  1, 61,
        60, 53, 65, 60, 52,  1, 67, 71, 56,  1, 56, 64, 58, 65, 62, 60, 52,  1,
        71, 63, 52,  1, 57, 56, 68, 52, 11,  1, 31, 60, 69,  1,  0, 52,  1, 54,
        65, 66, 60, 52,  1, 55, 65,  1, 55, 56, 69, 56, 64, 59, 65, 11,  1,  0,
         0,  0,  0, 30, 60, 75, 60, 52,  1, 65,  1, 62, 60, 72, 68, 65, 22,  1,
        

In [7]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([35,  1,  0,  0, 29, 56, 68, 70, 52])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([35]) the target: 1
when input is tensor([35,  1]) the target: 0
when input is tensor([35,  1,  0]) the target: 0
when input is tensor([35,  1,  0,  0]) the target: 29
when input is tensor([35,  1,  0,  0, 29]) the target: 56
when input is tensor([35,  1,  0,  0, 29, 56]) the target: 68
when input is tensor([35,  1,  0,  0, 29, 56, 68]) the target: 70
when input is tensor([35,  1,  0,  0, 29, 56, 68, 70]) the target: 52


In [10]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[52, 68, 65,  9,  1, 57, 52, 75],
        [ 1, 66, 65, 68,  1, 63, 60, 64],
        [54, 52, 69, 52,  2,  1,  0,  0],
        [68,  1, 55, 60, 52,  9,  1, 65]])
targets:
torch.Size([4, 8])
tensor([[68, 65,  9,  1, 57, 52, 75, 56],
        [66, 65, 68,  1, 63, 60, 64, 59],
        [52, 69, 52,  2,  1,  0,  0,  0],
        [ 1, 55, 60, 52,  9,  1, 65, 69]])
----
when input is [52] the target: 68
when input is [52, 68] the target: 65
when input is [52, 68, 65] the target: 9
when input is [52, 68, 65, 9] the target: 1
when input is [52, 68, 65, 9, 1] the target: 57
when input is [52, 68, 65, 9, 1, 57] the target: 52
when input is [52, 68, 65, 9, 1, 57, 52] the target: 75
when input is [52, 68, 65, 9, 1, 57, 52, 75] the target: 56
when input is [1] the target: 66
when input is [1, 66] the target: 65
when input is [1, 66, 65] the target: 68
when input is [1, 66, 65, 68] the target: 1
when input is [1, 66, 65, 68, 1] the target: 63
when input is [1, 66, 65, 

In [11]:
print(xb) # our input to the transformer

tensor([[52, 68, 65,  9,  1, 57, 52, 75],
        [ 1, 66, 65, 68,  1, 63, 60, 64],
        [54, 52, 69, 52,  2,  1,  0,  0],
        [68,  1, 55, 60, 52,  9,  1, 65]])


In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 80])
tensor(4.7658, grad_fn=<NllLossBackward0>)

TRa”b?-5V09;RGa2DOOM;s!"aBPf-5Ajxsh-7xO/X;!CL1*!C)EfPe>R608IO3Gcd¥DRLPGaChL*vd(G*aLrFJi!pqZ.9C=!—7z2


In [13]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [14]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

4.583566188812256


In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('/kaggle/input/pequeno-p-text/pequenop.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.211664 M parameters
step 0: train loss 4.6211, val loss 4.6277
step 100: train loss 2.4508, val loss 2.4307
step 200: train loss 2.3232, val loss 2.3055
step 300: train loss 2.2472, val loss 2.2137
step 400: train loss 2.1608, val loss 2.1433
step 500: train loss 2.0860, val loss 2.0706
step 600: train loss 2.0224, val loss 2.0220
step 700: train loss 1.9692, val loss 1.9860
step 800: train loss 1.9365, val loss 1.9539
step 900: train loss 1.8848, val loss 1.9147
step 1000: train loss 1.8519, val loss 1.8798
step 1100: train loss 1.8293, val loss 1.8694
step 1200: train loss 1.7961, val loss 1.8524
step 1300: train loss 1.7852, val loss 1.8331
step 1400: train loss 1.7413, val loss 1.8059
step 1500: train loss 1.7197, val loss 1.7891
step 1600: train loss 1.6927, val loss 1.7638
step 1700: train loss 1.6675, val loss 1.7572
step 1800: train loss 1.6313, val loss 1.7215
step 1900: train loss 1.6041, val loss 1.7239
step 2000: train loss 1.5837, val loss 1.7010
step 2100: train loss 1.

## Acknowledgements
---

[Google colab for the video](https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing#scrollTo=hoelkOrFY8bN)

[Let's build GPT: from scratch, in code, spelled out.](https://www.youtube.com/watch?v=kCc8FmEb1nY&t=1737s)

In regards to the original code, **there's only a minor change in the dataset used, with the rest copied and studied as a means to grasp the concepts** (within my limitations).